In [ ]:
import pandas as pd

from mainnet_launch.pages.risk_metrics.render_exit_liquidity_batch import (
    _load_quote_batch_options_from_db,
    _fetch_asset_allocation_from_db,
    _load_full_quote_batch_df,
    identify_suspect_exit_liquidity_quotes,
)
from mainnet_launch.database.postgres_operations import (
    get_full_table_as_df,
    _exec_sql_and_cache,
)
from mainnet_launch.constants.chains import ALL_CHAINS


def fetch_latest_50th_percentile_quote_batch_data():
    """Fetch the batch of quotes from our api and"""
    options = _load_quote_batch_options_from_db()
    latest_50th_percentile_option = [o for o in options if o["percent_exclude_threshold"] == 50][0]
    latest_batch_number = latest_50th_percentile_option["quote_batch"]
    asset_exposure_df = _fetch_asset_allocation_from_db(latest_batch_number)
    swap_quotes_df, median_reference_prices = _load_full_quote_batch_df(latest_batch_number)

    median_df = (
        swap_quotes_df.groupby(["api_name", "buy_token_symbol", "sell_token_symbol", "scaled_amount_in", "chain_id"])[
            "slippage_bps"
        ]
        .median()
        .reset_index()
    )
    median_df["bps_threshold"] = median_df["buy_token_symbol"].apply(lambda symbol: 50 if symbol == "WETH" else 25)

    above_threshold_df = median_df[median_df["slippage_bps"] > median_df["bps_threshold"]].copy()
    chain_id_to_name = {c.chain_id: c.name for c in ALL_CHAINS}
    above_threshold_df["chain_name"] = above_threshold_df["chain_id"].map(chain_id_to_name)
    asset_exposure_df = asset_exposure_df[["token_symbol", "reference_symbol", "quantity", "chain_id"]]
    return above_threshold_df, asset_exposure_df


def determine_maybe_over_exposed_assets(suspect_quotes_df: pd.DataFrame, asset_exposure_df: pd.DataFrame):
    df = pd.merge(
        suspect_quotes_df,
        asset_exposure_df,
        left_on=["chain_id", "sell_token_symbol"],
        right_on=["chain_id", "token_symbol"],
    )
    df["percent_sold_that_breaks_slippage_threshold"] = ((df["scaled_amount_in"] / df["quantity"]) * 100).round(2)
    df = df[df["quantity"] > df["scaled_amount_in"]]  # only look at currently over exposed

    maybe_over_exposed_df = (
        df.groupby(["api_name", "chain_name", "sell_token_symbol", "buy_token_symbol"])
        .agg(
            {
                "percent_sold_that_breaks_slippage_threshold": "min",
                "quantity": "first",
            }
        )
        .reset_index()
    )
    maybe_over_exposed_df["minimal_safe_sellable_quantity"] = maybe_over_exposed_df["quantity"] * (
        maybe_over_exposed_df["percent_sold_that_breaks_slippage_threshold"] / 100
    )

    return maybe_over_exposed_df


suspect_quotes_df, asset_exposure_df = fetch_latest_50th_percentile_quote_batch_data()
maybe_over_exposed_df = determine_maybe_over_exposed_assets(suspect_quotes_df, asset_exposure_df)
maybe_over_exposed_df

2025-11-04 11:45:02.724 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 11:45:02.725 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 11:45:02.725 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 11:45:02.726 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 11:45:02.726 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 11:45:02.726 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 11:45:02.727 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 11:45:02.727 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

,api_name,chain_name,sell_token_symbol,buy_token_symbol,percent_sold_that_breaks_slippage_threshold,quantity,minimal_safe_sellable_quantity
0,odos,base,GHO,USDC,5.86,"3,415,632.33","200,156.05"
1,odos,base,rETH,WETH,27.85,179.56,50.01
2,odos,eth,osETH,WETH,31.65,"1,421.84",450.01
3,odos,eth,pxETH,WETH,0.64,777.52,4.98
4,odos,eth,reUSD,DOLA,84.57,"1,418,970.65","1,200,023.48"
5,tokemak,base,GHO,USDC,5.86,"3,415,632.33","200,156.05"
6,tokemak,base,rETH,WETH,55.69,179.56,100.00
7,tokemak,eth,osETH,WETH,49.23,"1,421.84",699.97
8,tokemak,eth,pxETH,WETH,6.43,777.52,49.99
9,tokemak,eth,reUSD,DOLA,98.66,"1,418,970.65","1,399,956.45"
